In [ ]:
import tensorflow as tf

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

In [ ]:
with tf.name_scope('input'):
    # None -> batch size can be any size, 784 -> flattened mnist image
    x = tf.placeholder(tf.float32, shape=[None, 784], name="x-input") 
    # target 10 output classes
    y_ = tf.placeholder(tf.float32, shape=[None, 10], name="y-input")

with tf.name_scope('weights'):
    W = tf.Variable(tf.zeros((784, 10)))
    
with tf.name_scope('bias'):
    b = tf.Variable(tf.zeros(10))

## Softmax

\begin{equation}
\text{evidence}_i = \sum_j W_{i,~ j} x_j + b_i
\end{equation}

\begin{equation}
y = \text{softmax}(\text{evidence})
\end{equation}

\begin{equation}
\text{softmax}(x)_i = \frac{\exp(x_i)}{\sum_j \exp(x_j)}
\end{equation}

In [ ]:
with tf.name_scope('softmax'):
    evidence = tf.matmul(x, W) + b
    y = tf.nn.softmax(evidence)

## Cross Entropy

ค่าความแตกต่างของประเภทข้อมูลใน Class

\begin{equation}
-\frac{1}{m}\sum_{i=1}^m[h(x_i) * \log(y_i)]
\end{equation}

In [ ]:
with tf.name_scope('cross_Entropy'):
    cost = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))

## Gradient Descent

In [ ]:
with tf.name_scope('gradient_descent'):
    gradient_descent = tf.train.GradientDescentOptimizer(0.5).minimize(cost)

## Test Model Accuracy

In [ ]:
with tf.name_scope('accuracy'):
    correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

## Run Tensorboard Server

`tensorboard --logdir=/tmp/tensorflow/ --port 6006`

## Create Visualization Graph

In [ ]:
logs_path = "/tmp/tensorflow/"
writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())

## Create Log

In [ ]:
tf.summary.scalar("cost", cost)
tf.summary.scalar("accuracy", accuracy)
summary_op = tf.summary.merge_all()

## Train

In [ ]:
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()

for i in range(1000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    _, summary = sess.run([gradient_descent, summary_op], feed_dict={x: batch_xs, y_: batch_ys})
    writer.add_summary(summary, i)
print "done"
print(sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels}))